## Описание данных
### Признаки

- `id`: уникальный номер сообщения в системе twitter;
- `tdate`: дата публикации сообщения (твита);
- `tname`: имя пользователя, опубликовавшего сообщение;
- `ttext`: текст сообщения (твита);
- `ttype`: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
- `trep`: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
- `trtw`: число ретвитов
- `tfav`: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
- `tstcount`: число всех сообщений пользователя в сети twitter;
- `tfol`: количество фоловеров пользователя (тех людей, которые читают пользователя);
- `tfrien`: количество друзей пользователя (те люди, которых читает пользователь);
- `listcount`: количество листов-подписок в которые добавлен твиттер-пользователь.

In [36]:
import pandas as pd
import numpy as np
import nltk
import re

from datetime import datetime
from pymystem3 import Mystem

from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier
from catboost import Pool

In [2]:
columns = ['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtw', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount']

negative_df = pd.read_csv('./datasets/negative.csv', sep=';', names=columns)
positive_df = pd.read_csv('./datasets/positive.csv', sep=';', names=columns)

In [3]:
negative_duplicated_ids = negative_df.loc[negative_df['id'].duplicated() == True]
positive_duplicated_ids = positive_df.loc[positive_df['id'].duplicated() == True]

print(len(negative_duplicated_ids))
print(len(positive_duplicated_ids))

0
0


In [4]:
tweets_df = pd.concat([negative_df, positive_df])

del negative_df
del positive_df

In [5]:
tweets_df.set_index(['id'], inplace=True)
tweets_df.drop(labels='tname', axis=1, inplace=True)

In [6]:
tweets_df = tweets_df.sample(frac=0.1, random_state=123456)

In [7]:
display(tweets_df.info())
display(tweets_df.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22683 entries, 409919406907867136 to 409407608362377216
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tdate      22683 non-null  int64 
 1   ttext      22683 non-null  object
 2   ttype      22683 non-null  int64 
 3   trep       22683 non-null  int64 
 4   trtw       22683 non-null  int64 
 5   tfav       22683 non-null  int64 
 6   tstcount   22683 non-null  int64 
 7   tfol       22683 non-null  int64 
 8   tfrien     22683 non-null  int64 
 9   listcount  22683 non-null  int64 
dtypes: int64(9), object(1)
memory usage: 1.9+ MB


None

,tdate,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
id,,,,,,,,,,
409919406907867136,1386567377,@Maria_Brovko Может в сонник заглянуть? К чему...,1,0,0,0,1918,27,94,1
410869105265491968,1386793802,Закончим этот день предновогодним озорством ;)...,1,0,0,0,6020,114,189,1
410798792842039296,1386777039,"Мне Урупон шею свернет :-)\nЛента, я Вас любил...",1,0,0,0,19852,347,175,7
410738861644726272,1386762750,@_skylovesme_ посмотри мои твиты за сегодняшне...,1,0,0,0,28123,287,198,3
408910075903082496,1386326733,"Я так хотела побыстрее попробывать сырный суп,...",-1,0,0,0,218,7,6,0


In [8]:
tweets_df['tdate'] = pd.to_datetime(tweets_df['tdate'], unit='s')

In [9]:
tweets_df.loc[tweets_df['ttype'] == -1, 'ttype'] = 0

In [10]:
display(tweets_df.head(5))

,tdate,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
id,,,,,,,,,,
409919406907867136,2013-12-09 05:36:17,@Maria_Brovko Может в сонник заглянуть? К чему...,1,0,0,0,1918,27,94,1
410869105265491968,2013-12-11 20:30:02,Закончим этот день предновогодним озорством ;)...,1,0,0,0,6020,114,189,1
410798792842039296,2013-12-11 15:50:39,"Мне Урупон шею свернет :-)\nЛента, я Вас любил...",1,0,0,0,19852,347,175,7
410738861644726272,2013-12-11 11:52:30,@_skylovesme_ посмотри мои твиты за сегодняшне...,1,0,0,0,28123,287,198,3
408910075903082496,2013-12-06 10:45:33,"Я так хотела побыстрее попробывать сырный суп,...",0,0,0,0,218,7,6,0


### Очистка текста твита

Решил оставить только кириллицу. Наверное можно было вытащить еще смайлики в отдельный признак (положительный/негативный), не уверен что не ухудшит результат. 

In [11]:
m = Mystem()
    
def lemmatize(text):
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
        
    return lemm_text


def clear_text(text):
    reg = r'[^а-яА-ЯёЁ]'
    text = re.sub(reg, ' ', text)
    text = text.split()
    text = " ".join(text)
    
    return text

def text_process(text):
    proccesed_text = lemmatize(clear_text(text)) 
    
    return proccesed_text

In [12]:
tweets_df['ttext_clear'] = tweets_df['ttext'].apply(text_process)

In [13]:
# tweets_df.to_csv("./datasets/tweets_lemm.csv")

In [14]:
tweets_df.drop(labels='ttext', axis=1, inplace=True)

In [15]:
tweets_df.head(5)

,tdate,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount,ttext_clear
id,,,,,,,,,,
409919406907867136,2013-12-09 05:36:17,1,0,0,0,1918,27,94,1,может в сонник заглядывать к что этот чушь\n
410869105265491968,2013-12-11 20:30:02,1,0,0,0,6020,114,189,1,заканчивать этот день предновогодний озорство ...
410798792842039296,2013-12-11 15:50:39,1,0,0,0,19852,347,175,7,я урупон шея свертывать лента я вы любить с\n
410738861644726272,2013-12-11 11:52:30,1,0,0,0,28123,287,198,3,посмотреть мой твита за сегодняшний утро если ...
408910075903082496,2013-12-06 10:45:33,0,0,0,0,218,7,6,0,я так хотеть быстро попробывать сырный суп что...


In [16]:
scaler_columns = tweets_df.drop(columns=['tdate', 'ttext_clear', 'ttype']).columns.to_list()

scaler = StandardScaler()
tweets_df.loc[:, scaler_columns] = pd.DataFrame(scaler.fit_transform(tweets_df.loc[:, scaler_columns]), index=tweets_df.index, columns=scaler_columns)

Отмасштабировал численные признаки, для приведения к единому масштабу и устранения разброса.

In [17]:
features = tweets_df.drop(labels=['tdate', 'ttype'], axis=1)
target = tweets_df['ttype']

In [18]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.75, random_state=123456)

In [19]:
corpus_train = features_train['ttext_clear'].values.astype('U')
corpus_test = features_test['ttext_clear'].values.astype('U')

In [20]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(corpus_train)

tf_idf_test = count_tf_idf.transform(corpus_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thelie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создал матрицу с частотой слов для создания признака для модели.

In [21]:
tf_idf_train_df = pd.DataFrame(tf_idf_train.toarray(), index=features_train.index)
tf_idf_test_df = pd.DataFrame(tf_idf_test.toarray(), index=features_test.index)

features_train.drop(labels='ttext_clear', axis=1, inplace=True)
features_test.drop(labels='ttext_clear', axis=1, inplace=True)

/Users/thelie/.pyenv/versions/3.8.3/envs/data_science_env/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [22]:
features_train = features_train.join(tf_idf_train_df)
features_test = features_test.join(tf_idf_test_df)

In [23]:
log_model = LogisticRegression(n_jobs=-1, random_state=123456)

# log_f1_score = cross_val_score(log_model, tf_idf_train_df, target, scoring='f1', cv=5, n_jobs=-1)
log_model.fit(tf_idf_train_df, target_train)

LogisticRegression(n_jobs=-1, random_state=123456)

In [24]:
predicted = log_model.predict(tf_idf_test_df)

log_model_f1 = f1_score(target_test, predicted)
print(f'{type(log_model).__name__}: F1-score={log_model_f1}')

LogisticRegression: F1-score=0.678467373976142


In [25]:
log_model_2 = LogisticRegression(n_jobs=-1, random_state=123456)

# log_f1_score = cross_val_score(log_model, features_train, target, scoring='f1', cv=5, n_jobs=-1)
log_model_2.fit(features_train, target_train)

LogisticRegression(n_jobs=-1, random_state=123456)

In [26]:
predicted_2 = log_model_2.predict(features_test)

log_model_f1_2 = f1_score(target_test, predicted_2)
print(f'{type(log_model_2).__name__}: F1-score={log_model_f1_2}')

LogisticRegression: F1-score=0.6786677676851087


In [27]:
train_pool = Pool(features_train, target_train)
test_pool = Pool(features_test, target_test) 

cat_model = CatBoostClassifier(iterations=100000, early_stopping_rounds=2000, learning_rate=0.05, thread_count=-1, eval_metric='F1', random_state=123456)

In [29]:
cat_model.fit(train_pool, eval_set=test_pool, verbose=100)

0:	learn: 0.6889163	test: 0.6734907	best: 0.6734907 (0)	total: 94.6ms	remaining: 2h 37m 42s
100:	learn: 0.7147067	test: 0.6675011	best: 0.6756152 (1)	total: 2.4s	remaining: 39m 31s
200:	learn: 0.7432088	test: 0.6743581	best: 0.6756152 (1)	total: 4.67s	remaining: 38m 36s
300:	learn: 0.7748547	test: 0.6721621	best: 0.6772224 (244)	total: 6.94s	remaining: 38m 20s
400:	learn: 0.7981366	test: 0.6681232	best: 0.6772224 (244)	total: 9.24s	remaining: 38m 14s
500:	learn: 0.8172789	test: 0.6685680	best: 0.6772224 (244)	total: 11.5s	remaining: 38m 8s
600:	learn: 0.8348406	test: 0.6688774	best: 0.6772224 (244)	total: 13.8s	remaining: 38m 5s
700:	learn: 0.8542167	test: 0.6683071	best: 0.6772224 (244)	total: 16.1s	remaining: 38m 3s
800:	learn: 0.8692990	test: 0.6699692	best: 0.6772224 (244)	total: 18.5s	remaining: 38m 9s
900:	learn: 0.8826960	test: 0.6699361	best: 0.6772224 (244)	total: 20.8s	remaining: 38m 6s
1000:	learn: 0.8937622	test: 0.6707580	best: 0.6772224 (244)	total: 23.1s	remaining: 38m 4

Для метрики выбрал F1-меру, так как наиболее подходящую для задачи классификации и отражающую полноту и точность модели.

У обоих моделей низкий результат и не почти отличается друг от друга. Видимо результат больше зависит от способа обработки текста твитов, а не от самой модели классификации.

В отдельном ноутбуке результат с применением модели Bert.